<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/sarcasm_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import urllib

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f0e6de5e680>)

In [3]:
vocab_size = 10000
embedding_dim = 256
trunc_type='pre'
padding_type='post'
oov_tok = "<OOV>"

sentences = []
labels = []

In [4]:
import pandas as pd

df = pd.read_json('sarcasm.json')


In [5]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
df.sort_values(by="is_sarcastic", ascending=False)

,article_link,headline,is_sarcastic
21954,https://local.theonion.com/area-mans-biggest-a...,area man's biggest accomplishment not ever kil...,1
10347,https://politics.theonion.com/melania-trump-ho...,melania trump hosts state dinner in stunning b...,1
20977,https://local.theonion.com/opening-soda-bottle...,opening soda bottle inadvertently makes man loser,1
10326,https://entertainment.theonion.com/david-blain...,david blaine starves self of attention for 33 ...,1
10327,https://local.theonion.com/neighbor-bragging-a...,neighbor bragging about 20-pound box he fedexed,1
...,...,...,...
11465,https://www.huffingtonpost.com/entry/medical-m...,new proposal could make it easier to research ...,0
11462,https://www.huffingtonpost.com/entry/trump-nat...,trump's national security transition is 'a mes...,0
11461,https://www.huffingtonpost.com/entry/how-imani...,how imani boyette's love for basketball helped...,0
11460,https://www.huffingtonpost.com/entry/barack-ob...,barack obama is fourth president to put americ...,0


In [7]:
df = df[df['is_sarcastic'] > 0]

In [8]:
max_length = int(df['headline'].str.len().max()) + 1

In [9]:
sentences=list(df['headline'])


In [10]:
len(sentences)

11724

In [11]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer


In [12]:
from tensorflow.keras.layers import TextVectorization

In [13]:
text_vectorization = TextVectorization (ragged=True, output_mode="int")

In [14]:
text_vectorization.adapt(sentences)

In [21]:
total_words = len(text_vectorization.get_vocabulary()) + 1

In [15]:
vec_sentences = text_vectorization(sentences)

In [16]:
vec_sentences[12]

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([7009, 4350,  403, 6112, 2300,   54,   26, 1376])>

In [17]:
vec_sentences[2]

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([  423,  2830, 15052,   391,  3986,  1824,   125])>

In [18]:
input_sequences = []
for line in vec_sentences:
	for i in range(1, len(line)):
		n_gram_sequence = line[:i+1]
		input_sequences.append(n_gram_sequence)

In [20]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [22]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [24]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 38, 100)           1916100   
                                                                 
 bidirectional_1 (Bidirectio  (None, 300)              301200    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 19161)             5767461   
                                                                 
Total params: 7,984,761
Trainable params: 7,984,761
Non-trainable params: 0
_________________________________________________________________


In [28]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("1.h5", save_best_only=True)
]

In [29]:
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
3254/3254 [==============================] - 78s 21ms/step - loss: 8.3156 - accuracy: 0.0606
Epoch 2/100
3254/3254 [==============================] - 48s 15ms/step - loss: 7.5170 - accuracy: 0.0859
Epoch 3/100
3254/3254 [==============================] - 44s 13ms/step - loss: 6.4536 - accuracy: 0.1070
Epoch 4/100
3254/3254 [==============================] - 42s 13ms/step - loss: 5.5767 - accuracy: 0.1391
Epoch 5/100
3254/3254 [==============================] - 44s 13ms/step - loss: 5.0275 - accuracy: 0.1703
Epoch 6/100
3254/3254 [==============================] - 42s 13ms/step - loss: 4.7110 - accuracy: 0.1929
Epoch 7/100
3254/3254 [==============================] - 43s 13ms/step - loss: 4.5178 - accuracy: 0.2093
Epoch 8/100
3254/3254 [==============================] - 42s 13ms/step - loss: 4.3925 - accuracy: 0.2183
Epoch 9/100
3254/3254 [==============================] - 42s 13ms/step - loss: 4.3021 - accuracy: 0.2286
Epoch 10/100
3254/3254 [==============================]

In [35]:
print(text_vectorization.get_vocabulary()[11])

by


In [39]:
seed_text = "There goes"
next_words = 15

for _ in range(next_words):
	token_list = text_vectorization([seed_text])[0]
	token_list = sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	output_word = text_vectorization.get_vocabulary()[int(predicted)]
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 19ms/step
There goes straight from to be in the goddamn beautiful cinnamon roll over the goddamn beautiful cinnamon
